In [18]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [19]:
tf.keras.backend.clear_session()

In [28]:
NUM_CLASESS = 6
NUM_FILES = 10

In [33]:
def convert_parquet_to_npy(input_folder, output_folder):
    npy_output_folder = os.path.join(output_folder, 'npy_data')
    
    # Ensure the output directory exists
    os.makedirs(npy_output_folder, exist_ok=True)
    
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.parquet'):
                parquet_path = os.path.join(root, file)
                df = pd.read_parquet(parquet_path)
                eeg_data = df.to_numpy()
                relative_path = os.path.relpath(parquet_path, input_folder)
                
                # Create the corresponding directory structure in the npy_data folder
                output_subfolder = os.path.join(npy_output_folder, os.path.dirname(relative_path))
                os.makedirs(output_subfolder, exist_ok=True)
                np.save(os.path.join(output_subfolder, file.replace('.parquet', '.npy')), eeg_data)

def read_data(data_folder, num_files=None):
    """
    Read EEG data from .npy files in the specified data folder.

    Parameters:
    - data_folder (str): Path to the main data folder containing 'train' and 'test' subfolders.
    - num_files (int or None): Number of files to read from each subfolder. If None, all files will be read.

    Returns:
    - train (array[Tuple[np.ndarray, np.ndarray]]): List of tuples containing train EEG data.
    - test (array[Tuple[np.ndarray, np.ndarray]]): List of tuples containing test EEG data.
    - train_labels (pd.DataFrame): DataFrame containing train labels.
    - test_labels (pd.DataFrame): DataFrame containing test labels.
    """
    train_eeg_folder = os.path.join(data_folder, 'train_eegs')
    test_eeg_folder = os.path.join(data_folder, 'test_eegs')

    def read_npy_folder(folder_path, n_files=None):
        arrays = []
        files_to_read = os.listdir(folder_path)[:n_files] if n_files else os.listdir(folder_path)
        for file in files_to_read:
            if file.endswith('.npy'):
                file_path = os.path.join(folder_path, file)
                array = np.load(file_path)
                arrays.append(array)
        print(f"Read {len(arrays)} files from {folder_path}.")
        return arrays

    # Read EEG data
    train_eeg = read_npy_folder(train_eeg_folder, num_files)
    test_eeg = read_npy_folder(test_eeg_folder)

    train_labels = pd.read_csv(os.path.join(data_folder, 'train.csv'), nrows=num_files)
    test_labels = pd.read_csv(os.path.join(data_folder, 'test.csv'))

    return train_eeg, test_eeg, train_labels, test_labels

In [67]:
def preprocess_eeg(X_train_eeg, target_length=None):
    if target_length is not None:
        eeg_array = np.array(X_train_eeg)[:target_length].astype(np.float32)
    else:
        eeg_array = np.array(X_train_eeg).astype(np.float32)
    return eeg_array

def create_model(input_shape_eeg, num_classes=6):
    """Create a model that can be trained for variable duration
    EEG data.

    Args:
        input_shape_eeg : shape of one EEG sample
        input_shape_spectrogram : shape of one Spectrogram sample
        num_classes : 6 for seizure, lpd, gpd, lrda, grda, other

    Returns:
        keras model
    """
    
    model = tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, input_shape=input_shape_eeg[1:]),
        tf.keras.layers.LSTM(units=64, input_shape=input_shape_eeg[1:], return_sequences=True),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.LSTM(units=64),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

def lr_schedule(epoch, lr):
    if epoch % 10 == 0 and epoch > 0:
        return lr * 0.9
    return lr

In [ ]:
#convert_parquet_to_npy('data', 'data/npy_data')

In [129]:
train,_test,train_labels,_test_labels = read_data('data/npy_data/npy_data',num_files=NUM_FILES)

Read 10 files from data/npy_data/npy_data/train_eegs.
Read 1 files from data/npy_data/npy_data/test_eegs.


In [131]:
Xtrain = np.array(tf.keras.preprocessing.sequence.pad_sequences(train, padding='post', dtype='float32'))
labels = pd.read_csv('train.csv', nrows=NUM_FILES)

X_train, X_val, y_train, y_val = train_test_split(Xtrain, labels, test_size=0.2, random_state=42, shuffle=True)


In [132]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [133]:
label_encoder = LabelEncoder()
encoded_labels_train = label_encoder.fit_transform(y_train[:, 8])
encoded_labels_val = label_encoder.fit_transform(y_val[:, 8])

y_train = tf.keras.utils.to_categorical(encoded_labels_train, num_classes=NUM_CLASESS).astype('float32')
y_val = tf.keras.utils.to_categorical(encoded_labels_val, num_classes=NUM_CLASESS).astype('float32')

In [134]:
X_train[1].shape

(94000, 20)

In [135]:
BATCH_SIZE = 2
input_shape_eeg = (BATCH_SIZE,None, 20)
out_shape = NUM_CLASESS


In [136]:
X_train[1].shape

(94000, 20)

In [137]:
model = create_model(input_shape_eeg, out_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
model.compile(optimizer=optimizer, loss = tf.keras.losses.KLDivergence(), metrics=['accuracy'])


In [138]:
model.fit(X_train, y_train, epochs=1, batch_size=BATCH_SIZE)

2024-02-17 12:11:43.519237: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at cudnn_rnn_ops.cc:1771 : UNKNOWN: CUDNN_STATUS_NOT_SUPPORTED
in external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc(1782): 'cudnnSetRNNDataDescriptor( data_desc.get(), data_type, layout, max_seq_length, batch_size, data_size, seq_lengths_array, (void*)&padding_fill)'
2024-02-17 12:11:43.519287: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 10764520602590575001


UnknownError: Graph execution error:

Detected at node cond/CudnnRNNV3 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/haroon/miniconda3/envs/tf/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/haroon/miniconda3/envs/tf/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/haroon/miniconda3/envs/tf/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/haroon/miniconda3/envs/tf/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/haroon/miniconda3/envs/tf/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/haroon/miniconda3/envs/tf/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/haroon/miniconda3/envs/tf/lib/python3.11/asyncio/events.py", line 80, in _run

